In [92]:
%pip install pyspark


In [93]:
%pip install findspark

In [94]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project").getOrCreate()

In [120]:
import json
config_path = "C:/Users/ankita.khanna/Desktop/Project/config.json"
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

input_files = config['INPUT_FILENAME']
output_format = config['FILE_FORMAT']['Output']
output_path = config['OUTPUT_PATH']

charges_df = spark.read.csv(input_files['Charges'], header=True, inferSchema=True)
damages_df = spark.read.csv(input_files['Damages'], header=True, inferSchema=True)
endorse_df = spark.read.csv(input_files['Endorse'], header=True, inferSchema=True)
primary_person_df = spark.read.csv(input_files['Primary_Person'], header=True, inferSchema=True)
units_df = spark.read.csv(input_files['Units'], header=True, inferSchema=True)
restrict_df = spark.read.csv(input_files['Restrict'], header=True, inferSchema=True)


def get_output_path(problem_number):
    return output_path[problem_number]

def write_output(df,problem_number, format):
    return df.write.format(format).save(get_output_path(problem_number))

Problem 1 -Find the number of crashes (accidents) in which number of males killed are greater than 2?



In [96]:
from pyspark.sql.functions import count

df_deaths_per_crash = (
    primary_person_df.filter((primary_person_df["PRSN_GNDR_ID"] == 'MALE') &
                             (primary_person_df.DEATH_CNT == 1))
    .groupBy("CRASH_ID")
    .agg(count("*").alias("death_per_crash"))
)
print(df_deaths_per_crash.filter(df_deaths_per_crash.death_per_crash>2).count())

0


Problem 2-  How many two wheelers are booked for crashes? 

In [97]:
from pyspark.sql.functions import col, instr, lower

df_count_of_two_wheeler_booked = (units_df
.filter((instr(lower(units_df.VEH_BODY_STYL_ID), "motorcycle") > 0)
        | (units_df.UNIT_DESC_ID =="PEDALCYCLIST")
))

df_count_of_two_wheeler_booked.count()


948

Problem 3- Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.

Method -1

In [111]:
from pyspark.sql.functions import instr, lower, count

df_top_5_vehicles_without_airbag = (
    primary_person_df
    .join(units_df, ["CRASH_ID","UNIT_NBR"], "inner")
    .filter(
        (instr(lower(units_df.VEH_BODY_STYL_ID), "car") > 0) &
        (primary_person_df.PRSN_TYPE_ID == 'DRIVER') &
        (primary_person_df.DEATH_CNT == 1) &
        (primary_person_df.PRSN_AIRBAG_ID == 'NOT DEPLOYED') 
    )
    .groupBy(units_df.VEH_MAKE_ID)
    .agg(count("*").alias("Death_per_vehicle_make_without_airbags"))
    .orderBy(count("*").alias("Death_per_vehicle_make_without_airbags").desc())
)

df_top_5_vehicles_without_airbag.show()


+-------------+--------------------------------------+
|  VEH_MAKE_ID|Death_per_vehicle_make_without_airbags|
+-------------+--------------------------------------+
|       NISSAN|                                     4|
|    CHEVROLET|                                     3|
|        HONDA|                                     2|
|         FORD|                                     2|
|     CADILLAC|                                     1|
|MERCEDES-BENZ|                                     1|
|      PONTIAC|                                     1|
|        BUICK|                                     1|
|          KIA|                                     1|
|     CHRYSLER|                                     1|
+-------------+--------------------------------------+



Method 2

In [109]:
from pyspark.sql.functions import instr, lower, count, dense_rank, col
from pyspark.sql.window import Window

window=Window.orderBy(col("Death_per_vehicle_make_without_airbags").desc())

df_top_5_vehicles_without_airbag = (
    primary_person_df
    .join(units_df, ["CRASH_ID","UNIT_NBR"], "inner")
    .filter(
        (instr(lower(units_df.VEH_BODY_STYL_ID), "car") > 0) &
        (primary_person_df.PRSN_TYPE_ID == 'DRIVER') &
        (primary_person_df.DEATH_CNT == 1) &
        (primary_person_df.PRSN_AIRBAG_ID == 'NOT DEPLOYED') 
    )
    .groupBy(units_df.VEH_MAKE_ID)
    .agg(count("*").alias("Death_per_vehicle_make_without_airbags"))
    .withColumn("rank", dense_rank().over(window))
    .filter(col("rank") < 6)
    .drop("rank")
    )

df_top_5_vehicles_without_airbag.show(5)


+-----------+--------------------------------------+
|VEH_MAKE_ID|Death_per_vehicle_make_without_airbags|
+-----------+--------------------------------------+
|     NISSAN|                                     4|
|  CHEVROLET|                                     3|
|      HONDA|                                     2|
|       FORD|                                     2|
|   CADILLAC|                                     1|
+-----------+--------------------------------------+
only showing top 5 rows



Problem 4 -Determine number of Vehicles with driver having valid licences involved in hit and run? 

In [112]:
df_driver_without_driving_license = (
    primary_person_df
    .join(units_df, ["CRASH_ID", "UNIT_NBR"], "inner")
    .filter(
        (primary_person_df.PRSN_TYPE_ID == 'DRIVER') &
        (units_df.VEH_HNR_FL == 'Y') &
        (
            (primary_person_df.DRVR_LIC_TYPE_ID.isin("DRIVER LICENSE", "COMMERCIAL DRIVER LIC", "OCCUPATIONAL")) &
            (~primary_person_df.DRVR_LIC_CLS_ID.isin("UNLICENSED", "NA", "UNKNOWN"))
        )
    )
    .select("CRASH_ID", "UNIT_NBR")
    .distinct()
    .count()
)

print(df_driver_without_driving_license)


2450


Problem -5 -- Which state has highest number of accidents in which females are not involved? 

In [101]:
from pyspark.sql.functions import countDistinct, dense_rank, col
from pyspark.sql.window import Window

window = Window.orderBy(col("Non Women Driver Accident Cases").desc())

df_state_having_highest_accidents_without_women = (
    primary_person_df
    .filter((primary_person_df.PRSN_GNDR_ID != 'FEMALE')
            &(primary_person_df.PRSN_TYPE_ID == 'DRIVER'))
    .groupBy(primary_person_df.DRVR_LIC_STATE_ID)
    .agg(countDistinct("CRASH_ID").alias("Non Women Driver Accident Cases"))
    .withColumn("rank", dense_rank().over(window))
	.filter(col("rank") == 1)
	.drop("rank")
)


df_state_having_highest_accidents_without_women.show()


+-----------------+-------------------------------+
|DRVR_LIC_STATE_ID|Non Women Driver Accident Cases|
+-----------------+-------------------------------+
|            Texas|                          61022|
+-----------------+-------------------------------+



Problem -6 Which are the Top 3rd to 5th VEH_MAKE_IDs that contribute to a largest number of injuries including death

In [102]:

from pyspark.sql.functions import sum, col
from pyspark.sql.window import Window

window = Window.orderBy(col("Total_Casualties").desc())
df_top_vehicle_cat = (
    primary_person_df
    .join(units_df, ["CRASH_ID", "UNIT_NBR"], "inner")
    .filter(units_df.VEH_MAKE_ID != 'NA')
    .groupBy(units_df.VEH_MAKE_ID)
    .agg(sum(primary_person_df.TOT_INJRY_CNT).alias("Total_Injuries"), 
         sum(primary_person_df.DEATH_CNT).alias("Total_Deaths"))
    .withColumn("Total_Casualties", col("Total_Injuries") + col("Total_Deaths"))
    .withColumn("rank", dense_rank().over(window))
	.filter((col("rank") <6 ) & (col("rank")>2))
	.drop("rank")
)

df_top_vehicle_cat.show()

+-----------+--------------+------------+----------------+
|VEH_MAKE_ID|Total_Injuries|Total_Deaths|Total_Casualties|
+-----------+--------------+------------+----------------+
|     TOYOTA|          3335|          13|            3348|
|      DODGE|          2469|           8|            2477|
|     NISSAN|          2428|          13|            2441|
+-----------+--------------+------------+----------------+



Problem -7  For all the body styles involved in crashes, mention the top ethnic user group of each unique body style  

In [103]:
window= Window.partitionBy("VEH_BODY_STYL_ID").orderBy(col("Count_of_crashes").desc())
df_count_of_crashes = (primary_person_df.join(units_df, ["CRASH_ID","UNIT_NBR"], "inner")
            .filter((~units_df.VEH_BODY_STYL_ID.isin(["NA", "UNKNOWN", "NOT REPORTED", "OTHER  (EXPLAIN IN NARRATIVE)"]))
            &(~primary_person_df.PRSN_ETHNICITY_ID.isin(["NA", "UNKNOWN"])))
            .groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID")
            .agg(countDistinct("CRASH_ID").alias("Count_of_crashes"))
            .withColumn("rank", dense_rank().over(window))
            .filter(col("rank") == 1)
            .drop("rank", "Count_of_crashes")).show()

+--------------------+-----------------+
|    VEH_BODY_STYL_ID|PRSN_ETHNICITY_ID|
+--------------------+-----------------+
|           AMBULANCE|            WHITE|
|                 BUS|            BLACK|
|      FARM EQUIPMENT|            WHITE|
|          FIRE TRUCK|            WHITE|
|          MOTORCYCLE|            WHITE|
|NEV-NEIGHBORHOOD ...|            WHITE|
|PASSENGER CAR, 2-...|            WHITE|
|PASSENGER CAR, 4-...|            WHITE|
|              PICKUP|            WHITE|
|    POLICE CAR/TRUCK|            WHITE|
|   POLICE MOTORCYCLE|            WHITE|
|SPORT UTILITY VEH...|            WHITE|
|               TRUCK|            WHITE|
|       TRUCK TRACTOR|            WHITE|
|                 VAN|            WHITE|
|   YELLOW SCHOOL BUS|            BLACK|
+--------------------+-----------------+



Problem-8  : Among the crashed cars, what are the Top 5 Zip Codes with highest number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)

In [104]:
from pyspark.sql.functions import countDistinct, instr, lower, col, dense_rank
from pyspark.sql.window import Window

window = Window.orderBy(col("Count_of_crashes").desc())

df_drunk_and_drive_cases = (
    primary_person_df.join(units_df, ["CRASH_ID", "UNIT_NBR"], "inner")
    .filter(
        ((instr(lower(units_df.CONTRIB_FACTR_1_ID), "alcohol") > 0) |
        (instr(lower(units_df.CONTRIB_FACTR_P1_ID), "alcohol") > 0) |
        (instr(lower(units_df.CONTRIB_FACTR_2_ID), "alcohol") > 0))
        & (primary_person_df.DRVR_ZIP !='NULL')
    )
    .groupby("DRVR_ZIP")
    .agg(countDistinct("CRASH_ID").alias("Count_of_crashes"))
    .withColumn("rank", dense_rank().over(window))
    .filter(col("rank") < 6)
    .drop("rank")
)

df_drunk_and_drive_cases.show()


+--------+----------------+
|DRVR_ZIP|Count_of_crashes|
+--------+----------------+
|   76010|              52|
|   78521|              47|
|   78130|              41|
|   75067|              41|
|   78542|              38|
|   78550|              35|
+--------+----------------+



Problem 9 - Count of Distinct Crash IDs where No Damaged Property was observed and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance

In [105]:
df_no_damage_property = (
units_df.join(damages_df, "CRASH_ID", "inner")
.filter((((units_df.VEH_DMAG_SCL_1_ID).isin("DAMAGED 5", "DAMAGED 6", "DAMAGED 7 HIGHEST")) |
((units_df.VEH_DMAG_SCL_2_ID).isin("DAMAGED 5", "DAMAGED 6", "DAMAGED 7 HIGHEST"))) &
(damages_df.DAMAGED_PROPERTY == "NONE") &
(units_df.FIN_RESP_TYPE_ID.isin("LIABILITY INSURANCE POLICY", "PROOF OF LIABILITY INSURANCE")))
.agg(countDistinct("CRASH_ID").alias("Count_of_crashes"))
                         )

df_no_damage_property.show()


+----------------+
|Count_of_crashes|
+----------------+
|               8|
+----------------+



Problem 10  Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers, used top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)

In [106]:
from pyspark.sql.functions import countDistinct, col, dense_rank
from pyspark.sql.window import Window
#  top 25 list of citites having highest number of offences
window = Window.orderBy(col("Count_of_crashes_having_charges").desc())

top_25_states_df = (
    primary_person_df.join(charges_df, ["CRASH_ID", "PRSN_NBR", "UNIT_NBR"], "inner")
    .filter(~primary_person_df.DRVR_LIC_STATE_ID.isin("Unknown", "NA", "Other"))
    .groupBy("DRVR_LIC_STATE_ID")
    .agg(countDistinct(charges_df.CRASH_ID).alias("Count_of_crashes_having_charges"))
    .withColumn("rank", dense_rank().over(window))
    .filter(col("rank") < 26)
    .drop("rank", "Count_of_crashes_having_charges")
)

list_of_top_states = list(row.DRVR_LIC_STATE_ID for row in top_25_states_df.collect())
print(list_of_top_states)



['Texas', 'Mexico', 'Louisiana', 'New Mexico', 'California', 'Florida', 'Oklahoma', 'Arkansas', 'Arizona', 'Georgia', 'Illinois', 'Colorado', 'Mississippi', 'Missouri', 'Tennessee', 'North Carolina', 'Kansas', 'Alabama', 'Ohio', 'Michigan', 'New York', 'Washington', 'Virginia', 'Nevada', 'Indiana', 'South Carolina', 'Pennsylvania']


In [107]:
#used top 10 used vehicle colours with highest number of offences
window = Window.orderBy(col("Count_of_crashes_having_charges").desc())
top_10_colours = (
    units_df.join(charges_df, ["CRASH_ID", "UNIT_NBR"], "inner")
    .filter(~units_df.VEH_COLOR_ID.isin("NA"))
    .groupBy("VEH_COLOR_ID")
    .agg(countDistinct(charges_df.CRASH_ID).alias("Count_of_crashes_having_charges"))
    .withColumn("rank", dense_rank().over(window))
    .filter(col("rank") < 11)
    .drop("rank", "Count_of_crashes_having_charges")
)
list_of_top_colours = list(row.VEH_COLOR_ID for row in top_10_colours.collect())
print(list_of_top_colours)


['WHI', 'BLK', 'SIL', 'GRY', 'BLU', 'RED', 'GRN', 'MAR', 'TAN', 'GLD']


In [108]:
window = Window.orderBy(col("Count_of_crashes_having_charges").desc())

df_top_5_makers=(
    primary_person_df.join(charges_df, ["CRASH_ID", "PRSN_NBR", "UNIT_NBR"], "inner")
    .join(units_df, ["CRASH_ID", "UNIT_NBR"], "inner")
    .filter((
            (primary_person_df.DRVR_LIC_TYPE_ID.isin("DRIVER LICENSE", "COMMERCIAL DRIVER LIC", "OCCUPATIONAL")) |
            (~primary_person_df.DRVR_LIC_CLS_ID.isin("UNLICENSED", "NA", "UNKNOWN")))&
            (instr(lower(charges_df.CHARGE),"speed") > 0) &
            (units_df.VEH_COLOR_ID.isin(list_of_top_colours)) &
            (units_df.VEH_LIC_STATE_ID.isin(list_of_top_states)))
    .groupby("VEH_MAKE_ID")
    .agg(countDistinct(charges_df.CRASH_ID).alias("Count_of_crashes_having_charges"))
    .withColumn("rank", dense_rank().over(window))
    .filter(col("rank") < 6)
    .drop("rank", "Count_of_crashes_having_charges")
    ).show()

+-----------+
|VEH_MAKE_ID|
+-----------+
+-----------+

